# Demo the Liquidator Agent Logic

This notebook:

1. Fetches the latest crvusd contract data into Python objects using `crvusdsim`.
2. Fetches prices and `ExternalMarket`s from the `baseline` scenario.
3. Instantiates a liquidator agent.
4. Checks if there are users to liquidate in the `Controller` and liquidates them.

TODO describe the liquidation logic in detail.

In [1]:
from src.sim.scenario import Scenario

%load_ext autoreload
%autoreload 2

In [2]:
# Generate markets and prices
scenario = Scenario("baseline", "wsteth")
sample = scenario.pricepaths[0]
scenario.prepare_for_run()  # Set block timestamps
scenario.prepare_for_trades(sample)  # Set External Market Price

# Unpack
markets = scenario.markets
liquidator = scenario.liquidator
controller = scenario.controller

[INFO][19:06:17][src.sim.scenario]-17150: Fetching sim_market from subgraph.


[INFO][19:06:26][src.utils.poolgraph]-17150: Found 20 valid trading cycles of length 3.


In [3]:
# Artificially inflate collateral price
collateral = scenario.controller.COLLATERAL_TOKEN.address
sample = scenario.pricepaths[0]
prices_usd = sample.prices_usd
assert collateral in prices_usd.keys()
prices_usd[collateral] *= 2
sample.update(prices_usd)
scenario.update_market_prices(sample)

In [4]:
profit, count = liquidator.perform_liquidations(controller)
print(f"Liquidated {count} positions for a profit of {profit} USD.")

Liquidated 25 positions for a profit of 3782287.5075479997 USD.


In [5]:
controller.users_to_liquidate()

In [12]:
for position in controller.users_to_liquidate():
    print(position.__dict__)

{'user': '0xfa4fc4ec2f81a4897743c5b4f45907c02ce06199', 'x': 0, 'y': 206336926670773, 'debt': 542236491615260053, 'health': -87155436831522523}
{'user': '0x36f19ac3719e29dffbd285d61e08ea00781b9789', 'x': 67676550063052644783815, 'y': 0, 'debt': 71324582120585382441215, 'health': -51146911051861024}
{'user': '0x101627601427770aed741f90dc160689d583f8fa', 'x': 0, 'y': 1786251842541253162, 'debt': 9796294144910721702528, 'health': -548021241217127902}


### A Quant Might Ask

Why are so many users eligible for liquidation? Wouldn't they already have been liquidated?

Yes. This is a known issue that is rooted in the crvusd subgraph. User positions are loaded in via two pathways:
1. We get `userState` snapshots from the subgraph, which snapshots a user's `depositedCollateral`, and `debt`. We use these values to load in their positions. Particularly the debt.
2. We get `bands` snapshots from the subgraph (a separate process at a different time). We use the relative `depositedCollateral` of each user to distribute the `stablecoin` and `collateral` from the `bandsSnapshot` to each user.

In theory, step (2) would correctly load in each user's share of `x,y` in LLAMMA so that we may correctly calculate their health and simulate their position. **HOWEVER** this is not the case, because of a bug in the subgraph. The bug is described in detail in this issue: https://github.com/curvefi/volume-subgraphs/issues/47.

At a high-level, bands with negative indices are being snapshot incorrectly due to the way negative integers are handled by the `graph-protocol` hexadecimal string class. When converting negative ints to hexadecimal strings, we are accidentally converting them into the positive representation. This means that if band `1` has 0 collateral and 0 stablecoin, and band `-1` has 1M collateral and 0 stablecoin, the snapshot will incorrectly tell us that band `-1` has 0, 0 instead of 1M, 0. 

This is what makes so many users *seem* like they are underwater.